# Jacobi Rotations for the Pipek-Mezey localization Method

- Defining the functional

$P = \sum_i \sum_A (q_i^A)^2$

We define a pair wise rotation between two orbitals $|i\rangle$ and $|j\rangle$ such that

$C_{\mu \tilde{i}} = \cos(\theta)C_{\mu i} + \sin(\theta)C_{\mu j}$

$C_{\mu \tilde{j}} = -\sin(\theta)C_{\mu i} + \cos(\theta)C_{\mu j}$

Maximizing $P$ leads to

$\frac{dP}{d\theta} = 0$

For which only terms involving orbitals $\tilde{i}$ and $\tilde{j}$ remain

$\frac{dP}{d\theta} = \sum_A \frac{d}{d\theta}(q_\tilde{i}^A)^2  + \frac{d}{d\theta}(q_\tilde{j}^A)^2$

$q_\tilde{i}^A = \sum_{\mu \in A} \sum_\nu C_{\mu \tilde{i}}\cdot C_{\tilde{i}\nu} \cdot S_{\nu \mu}$ 

$q_\tilde{i}^A = \sum_{\mu \in A} \sum_\nu (\cos(\theta)C_{\mu i} + \sin(\theta)C_{\mu j})\cdot (\cos(\theta)C_{i \nu} + \sin(\theta)C_{j \nu}) \cdot S_{\nu \mu}$ 

$q_\tilde{i}^A = \cos^2(\theta)\sum_{\mu \in A} \sum_\nu C_{\mu i} C_{i \nu} S_{\nu \mu} + \sin^2(\theta)\sum_{\mu \in A} \sum_\nu C_{\mu j} C_{j \nu} S_{\nu \mu} + \cos(\theta)\sin(\theta)\sum_{\mu \in A} \sum_\nu (C_{\mu i} C_{j \nu} S_{\nu \mu} + C_{\mu j} C_{i \nu} S_{\nu \mu})$

### Define

$Q^A_i = \sum_{\mu \in A}\sum_\nu C_{\mu i} C_{i \nu} S_{\nu \mu}$

$Q^A_j = \sum_{\mu \in A}\sum_\nu C_{\mu j} C_{j \nu} S_{\nu \mu}$

$Q^A_{ij} = \sum_{\mu \in A} \sum_\nu (C_{\mu i} C_{j \nu} S_{\nu \mu} + C_{\mu j} C_{i \nu} S_{\nu \mu})$

$q_\tilde{i}^A = \cos^2(\theta)Q^A_{i} + \sin^2(\theta)Q^A_{j} + \cos(\theta)\sin(\theta)Q^A_{ij}$

In [6]:
using Symbolics

In [7]:
@variables Qi Qj Qij θ

4-element Vector{Num}:
  Qi
  Qj
 Qij
   θ

In [17]:
qia = Qi*cos(θ)^2 + Qj*sin(θ)^2 + cos(θ)*sin(θ)*Qij

Qi*(cos(θ)^2) + Qj*(sin(θ)^2) + Qij*cos(θ)*sin(θ)

In [15]:
qja = Qj*cos(θ)^2 + Qi*sin(θ)^2 - cos(θ)*sin(θ)*Qij

Qi*(sin(θ)^2) + Qj*(cos(θ)^2) - Qij*cos(θ)*sin(θ)

In [12]:
∂ = Differential(θ)

(::Differential) (generic function with 2 methods)

In [24]:
dP = ∂(qia^2) + ∂(qja^2) |> expand_derivatives

(2Qi*(cos(θ)^2) + 2Qj*(sin(θ)^2) + 2Qij*cos(θ)*sin(θ))*(Qij*(cos(θ)^2) + 2Qj*cos(θ)*sin(θ) - Qij*(sin(θ)^2) - 2Qi*cos(θ)*sin(θ)) + (2Qi*(sin(θ)^2) + 2Qj*(cos(θ)^2) - 2Qij*cos(θ)*sin(θ))*(Qij*(sin(θ)^2) + 2Qi*cos(θ)*sin(θ) - Qij*(cos(θ)^2) - 2Qj*cos(θ)*sin(θ))

In [161]:
(qia*∂(qia) + qja*∂(qja)) |> expand_derivatives |> expand |> simplify

Qij*Qj*(-(cos(θ)^4) - (sin(θ)^4)) + Qi*Qij*(cos(θ)^4 + sin(θ)^4) + 2(Qi^2)*(sin(θ)^3)*cos(θ) + 2(Qj^2)*(sin(θ)^3)*cos(θ) + 2(Qij^2)*(cos(θ)^3)*sin(θ) + 6Qij*Qj*(cos(θ)^2)*(sin(θ)^2) + 4Qi*Qj*(cos(θ)^3)*sin(θ) - 2(Qi^2)*(cos(θ)^3)*sin(θ) - 2(Qij^2)*(sin(θ)^3)*cos(θ) - 2(Qj^2)*(cos(θ)^3)*sin(θ) - 4Qi*Qj*(sin(θ)^3)*cos(θ) - 6Qi*Qij*(cos(θ)^2)*(sin(θ)^2)

In [180]:
handsimp = Qij*(Qi-Qj)*cos(4θ) + (Qij^2 - (Qi - Qj)^2)*sin(4θ)/2

Qij*(Qi - Qj)*cos(4θ) + (1//2)*(Qij^2 - ((Qi - Qj)^2))*sin(4θ)

### Verify numerically the equivalent between the raw equation and the hand simplified one

In [206]:
N = 100
xvals = zeros(N)
for θval = collect(2π*(0:N)/N)
    qival = randn()
    qjval = randn()
    qijval = randn()
    x = substitute(dP, Dict([θ => θval, Qi=>qival, Qj=>qjval, Qij=>qijval]))
    y = substitute(handsimp, Dict([θ => θval, Qi=>qival, Qj=>qjval, Qij=>qijval]))
    xvals[N] = abs(x.val - 2*y.val)
end
maximum(xvals)

2.7755575615628914e-17

### Define

$A = 2\sum_{A} Q^A_{ij}(Q^A_i - Q^A_j)$

$B = \sum_A (Q^A_{ij})^2 - (Q^A_i - Q^A_j)^2$

In [185]:
function A(Qij, Qi, Qj)
    N = length(Qij)
    2*sum(
        Qij[i]*(Qi[i] - Qj[i]) for i = 1:N
    )
end

A (generic function with 1 method)

In [186]:
function B(Qij, Qi, Qj)
    N = length(Qij)
    sum(
        Qij[i]^2 - (Qi[i] - Qj[i])^2 for i = 1:N
    )
end

B (generic function with 1 method)

In [246]:
N = 1000
qival = rand(N)
qjval = rand(N)
qijval = rand(N)
Aval = A(qijval, qival, qjval)
Bval = B(qijval, qival, qjval)
α = atan(-Aval/Bval)/4
s = 0.0
for i = 1:N
    x = substitute(dP, Dict([θ => α, Qi=>qival[i], Qj=>qjval[i], Qij=>qijval[i]]))
    s += x.val
end
s

-4.801714581503802e-15